# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [2]:
#Your code here
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

%matplotlib inline

In [3]:
df = pd.read_csv('homepage_actions.csv')
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


In [5]:
df.action.value_counts()

view     6328
click    1860
Name: action, dtype: int64

In [6]:
df.id.value_counts()

750053    2
389447    2
283040    2
918794    2
208714    2
         ..
467818    1
629924    1
820336    1
424916    1
461199    1
Name: id, Length: 6328, dtype: int64

In [23]:
# Use set to get unique id values
clicks = set(df[df.action == 'click']['id'])
views = set(df[df.action == 'view']['id'])

In [29]:
# Using set math determine if there were clickers who didn't view, and vice verso

click_no_view = clicks - views
view_no_click = views - clicks
print('There were a total of {} views, and {} clicks.'.format(len(views), len(clicks)))
print("There were {} viewers who didn't click.".format(len(view_no_click)))
print("There were {} clickers who didn't view.".format(len(click_no_view)))

There were a total of 6328 views, and 1860 clicks.
There were 4468 viewers who didn't click.
There were 0 clickers who didn't view.


In [32]:
# Check for overlap in control and experimental group
control = set(df[df.group == 'control']['id'])
experiment = set(df[df.group == 'experiment']['id'])
unique_id = set(df.id)

interlap = (len(control) + len(experiment)) - len(unique_id)

print('There were a total of {} unique participants in this experiment.'.format(len(unique_id)))
print('There were a total of {} participants in the control group.'.format(len(control)))
print('There were a total of {} participants in the experiment group.'.format(len(experiment)))
print("That means there were {} amount of overlapping id's.".format(interlap))

There were a total of 6328 unique participants in this experiment.
There were a total of 3332 participants in the control group.
There were a total of 2996 participants in the experiment group.
That means there were 0 amount of overlapping id's.


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [ ]:
'''
Null Hypothesis: The experimental homepage was as or less affective as the original page.
Alternative Hypothesis: The experimental page improved the rate of clicks compared to the original page.

Confidence Level alpha = .05
'''

In [44]:
#Your code here
control_group = df[df.group == 'control']
experiment_group = df[df.group == 'experiment']

control_view_no_click = control - set(control_group[control_group.action == 'click']['id']) 
print("{} out of {} users in the control group didn't click".format(len(control_view_no_click), len(control)))

experiment_view_no_click = experiment - set(experiment_group[experiment_group.action == 'click']['id']) 
print("{} out of {} users in the experiment group didn't click".format(len(experiment_view_no_click), len(experiment)))

2400 out of 3332 users in the control group didn't click
2068 out of 2996 users in the experiment group didn't click


In [53]:
contingency_table = np.array([(len(control_view_no_click), len(experiment_view_no_click)),
                           (len(control-control_view_no_click), len(experiment-experiment_view_no_click))])
print(contingency_table)

[[2400 2068]
 [ 932  928]]


In [55]:
results = stats.contingency.chi2_contingency(contingency_table)
print(results)

(6.712921132285345, 0.009571680497042266, 1, array([[2352.61946903, 2115.38053097],
       [ 979.38053097,  880.61946903]]))


In [63]:
print('Our p-value comes out to {}, much lower than our confidence level of .05'.format(round(results[1], 5)))

control_click_rate = len(control-control_view_no_click)/len(control_group)
experiment_click_rate = len(experiment-experiment_view_no_click)/len(experiment_group)

print('Click percentage control group {}%'.format(100*round(control_click_rate, 4)))
print('Click percentage experiment group {}%'.format(100*round(experiment_click_rate, 4)))
print('Click rate increase: {}%'.format(100*round(experiment_click_rate-control_click_rate, 4)))

Our p-value comes out to 0.00957, much lower than our confidence level of .05
Click percentage control group 21.86%
Click percentage experiment group 23.65%
Click rate increase: 1.79%


In [ ]:
'''
With this we can confidently reject the null hypothesis.
There is a clear improvement on clicks per view using the experimental homepage
of around 1.79% up from 21.86%.
'''

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [66]:
#Your code here
expected_exp_clicks = len(experiment_group) * control_click_rate
print('Expected clicks in experiment group: {}'.format(expected_exp_clicks))

Expected clicks in experiment group: 857.6848030018762


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [69]:
#Your code here
n = len(control_group)
var = n*control_click_rate*(1-control_click_rate)
std = np.sqrt(var)
num_of_std = (len(experiment-experiment_view_no_click) - expected_exp_clicks)/std

print('Number of standard deviations away: {}'.format(round(num_of_std, 2)))

Number of standard deviations away: 2.61


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [71]:
#Your code here
p_value = stats.norm.sf(num_of_std)
print('Resulting p-value: {}'.format(p_value))

Resulting p-value: 0.004586510360224532


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: My result was twice as large as this result, but still much smaller than .005. 
This might have been because I utilized a chi-squared test instead of a t-test. 

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.